<div style="color:#2db099">
Um festzustellen, in welchem Verhältnis von Bots bzw. normalen Nutzern gesetzten Pixeln zensiert wurden, 

soll im folgenden erstmal für einen Datensatz alle Pixel bestimmt werden, die vom Zensur-Quadrat betroffen sind.
</div>

In [1]:
from src.data.dataset_functions import get_dataframei_onlymods
import pyspark.sql.functions as F

mods_dataframe = get_dataframei_onlymods(4)

providing ../data/raw/2022_place_canvas_history-000000000004.csv ...
../data/raw/2022_place_canvas_history-000000000004.csv is already in data/raw
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9ff87d56-a853-4c6b-928d-03e9baa5dc36;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 72ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

In [2]:
mods_dataframe.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- x1: integer (nullable = true)
 |-- y1: integer (nullable = true)
 |-- x2: integer (nullable = true)
 |-- y2: integer (nullable = true)
 |-- t: long (nullable = true)
 |-- pixel_color: string (nullable = true)



In [3]:
import pandas as pd

#erstellt Pandas DataFrame mit allen Pixel des Zensur-Quadrates
mods_dataframe = mods_dataframe.drop('pixel_color').collect()
pandasMB = pd.DataFrame() #Neues DataFrame erstellen, dass befüllt werden soll
pandasMB.insert(0,"user_id", [], True)
pandasMB.insert(1,"x", [], True)
pandasMB.insert(2,"y", [], True)
pandasMB.insert(3,"t", [], True)
for row in mods_dataframe:
    user_id,x1,y1,x2,y2,t = list(row)
    for i in range(x1,x2+1): #zunächst obere Rahmenlinie erstellen
        #pandasMB.loc[len(pandasMB.index)] = [user_id,i,y1,t]
        for j in range(y1,y2+1):
            pandasMB.loc[len(pandasMB.index)] = [user_id,i,j,t]

In [4]:
#Jetzt Pandas DataFrame zum Spark DataFrame umwandeln
#from pyspark.sql import Row
from pyspark.sql import SparkSession
 
# creating the session
spark = SparkSession.builder.getOrCreate()

df = spark.createDataFrame(pandasMB)
df.show()

+--------------------+---+----+----------+
|             user_id|  x|   y|         t|
+--------------------+---+----+----------+
|gXOuxCyb/dslyRb5G...| 23|1523|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1524|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1525|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1526|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1527|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1528|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1529|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1530|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1531|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1532|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1533|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1534|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1535|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1536|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1537|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1538|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1539|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1540|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1541|1649045540|
|gXOuxCyb/dslyRb5G...| 23|1542|1649045540|
+----------

<div style="color:#2db099">
Damit wäre das DataFrame mit den zensierten Pixeln fertig!
</div>

<div style="color:#2db099">
Jetzt muss man die Funktion erstellen, die rückwirkend schaut, welcher Pixel von der Zensur denn nun betroffen ist und diesen in ein eigenes DataFrame packen als Ausgabe.
</div>

In [5]:
pandasZensiertes = pd.DataFrame() #neues DataFrame anlegen für betroffene Pixel
pandasZensiertes.insert(0,"user_id", [], True)
pandasZensiertes.insert(1,"x", [], True)
pandasZensiertes.insert(2,"y", [], True)
pandasZensiertes.insert(3,"t", [], True)
pandasZensiertes.insert(4,"pixel_color", [], True)

In [17]:
#Jetzt die Funktion fürs Finden der entsprechenden Pixel
#df1 ist DataFrame mit ZensurPixel
#df2 DataFrame mit normalen Nutzer Daten die untersucht werden sollen
def censoredPixels(df1,df2):
    '''Funktion liefert die Pixel die zensiert wurden'''
    cenPix = df1.collect()
    for row in cenPix:
        user_id,x,y,t = list(row)
        #in Zeit zurückschauen
        for i in range(t,0,-1):
            #select_rows(df,['apple','banana']) | df[(df.values  == "banana")|(df.values  == "apple" ) ]
            #user_id2,x2,y2,t2,pixel_color = list(df2[(df2.x == x)(df2.y == y)(df2.t == t)])
            #sqlc.sql('select * from students where roll_no=%s and name="%s"' % ('1', 'ravindra')).show()
            #df_filtered = df[(df['Shares']>=100) & (df['Shares']<=150)]
            hold = df2[(df2['x']==x) & (df2['y']==y) & (df2['t']==t)]
            hold = hold.collect()
            if hold:
            #hold = df2.where(df2.x == x & df2.y == y & df2.t == t).collect()
                user_id2,x2,y2,t2,pixel_color = list(hold)
            #user_id2,x2,y2,t2,pixel_color = list(df2.where('x ="%s" AND y ="%s" AND t ="%s"' % (x,y,t)).collect())
                if(not(user_id2 is None and x2 is None and y2 is None and t2 is None and pixel_color is None)):
                    pandasZensiertes.loc[len(pandasZensiertes.index)] = [user_id2,x2,y2,t2,pixel_color]
                    break
    return pandasZensiertes

In [7]:
from src.data.dataset_functions import get_dataframe
#DataFrame mit normalen Nutzern generieren
dataFrame = get_dataframe(True)
#dateFrame = dataFrame.toPandas()

providing ../data/raw/2022_place_canvas_history-000000000004.csv ...
../data/raw/2022_place_canvas_history-000000000004.csv is already in data/raw


In [18]:
#DataFrame mit betroffenen Pixeln erhalten
censored = spark.createDataFrame(censoredPixels(df,dataFrame))

ERROR:root:KeyboardInterrupt while sending command.               (24 + 4) / 35]
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 